In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mahitha1111","key":"b97b90e5726a3ba371689d223ff9c9db"}'}

In [ ]:
! pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 5.4MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=7d898de6e8569669b6acde171fa561d8dd0c1eda008834faa01871a69a68342b
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c deep-learning-hw2p2-part-2-verification

 94% 97.0M/103M [00:02<00:00, 34.8MB/s]
100% 103M/103M [00:02<00:00, 37.9MB/s] 


In [ ]:
!unzip \*.zip  && rm *.zip

Streaming output truncated to the last 5000 lines.
  inflating: verification_data/00064100.jpg  
  inflating: verification_data/00064101.jpg  
  inflating: verification_data/00064102.jpg  
  inflating: verification_data/00064103.jpg  
  inflating: verification_data/00064104.jpg  
  inflating: verification_data/00064105.jpg  
  inflating: verification_data/00064106.jpg  
  inflating: verification_data/00064107.jpg  
  inflating: verification_data/00064108.jpg  
  inflating: verification_data/00064109.jpg  
  inflating: verification_data/00064110.jpg  
  inflating: verification_data/00064111.jpg  
  inflating: verification_data/00064112.jpg  
  inflating: verification_data/00064113.jpg  
  inflating: verification_data/00064114.jpg  
  inflating: verification_data/00064115.jpg  
  inflating: verification_data/00064116.jpg  
  inflating: verification_data/00064117.jpg  
  inflating: verification_data/00064118.jpg  
  inflating: verification_data/00064119.jpg  
  inflating: verification_dat

In [ ]:
import os
import numpy as np
from PIL import Image
from torchvision import transforms
import torch
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Here wer are accessing the two images at a time
class MyDatasetVerify(Dataset):
    # Create dataset for validation of verification test
    def __init__(self, pairFN, imgFolderPath):
        self.imgFolderPath = imgFolderPath
        with open(pairFN) as f:
            self.pairList = [line.rstrip() for line in f]
    def __len__(self):
        return len(self.pairList)
    def __getitem__(self, idx):
        items = self.pairList[idx].split()
        fn1, fn2 = items[0], items[1]
        img1 = PIL.Image.open(self.imgFolderPath + fn1)
        img2 = PIL.Image.open(self.imgFolderPath + fn2)
        img1 = transforms.ToTensor()(img1)
        img2 = transforms.ToTensor()(img2)
        if len(items) == 3: # validation
            return img1, img2, int(items[2])
        else: # test
            return img1, img2, -1
    def getPairList(self):
        return self.pairList

In [ ]:

import time
import PIL
def testVerify(model, vLoader):
    similarity = np.array([])
    true = np.array([])
    res=[]
    start_time = time.time()
    with torch.no_grad():
        start_time = time.time()
        for batch_idx, (imgs1, imgs2, targets) in enumerate(vLoader):

            imgs1, imgs2, targets = imgs1.cuda(), imgs2.cuda(), targets.cuda()
            # find cos similarity between embeddings
            imgs1Embed = model(imgs1.float())
            imgs2Embed = model(imgs2.float())
            # print(imgs1Embed.shape)

            sim = F.cosine_similarity(imgs1Embed, imgs2Embed,dim=1) 
            print(sim.shape)
            similarity = np.concatenate((similarity, sim.cpu().numpy().reshape(-1)))
            # print(similarity)
            true = np.concatenate((true, targets.cpu().numpy().reshape(-1)))
            
            del imgs1
            del imgs2
            del targets
    return similarity, true,res

In [ ]:
# Read in verification pairs for validation
hyper={
    "verifyPairListPath":'/content/verification_pairs_val.txt',
    "verifyImgFolderPath":'/content/',
    "verifyTestPairListPath":'/content/verification_pairs_test.txt',
    "verifyTestImgFolderPath":'/content/',
    
}
verifyData_valid = MyDatasetVerify(hyper["verifyPairListPath"], hyper["verifyImgFolderPath"])
verify_loader_args_valid = dict(shuffle=False, batch_size=200, num_workers=2, pin_memory=True)
verify_loader_valid = DataLoader(verifyData_valid, **verify_loader_args_valid)

In [ ]:
# Load the network with torch.load
path='/content/drive/MyDrive/turnkey/CNN-Model9'
state_dict = torch.load(path)
# #print(len(state_dict["net.14.weight"]))
network.load_state_dict(state_dict)


<All keys matched successfully>

In [ ]:

# Calculate simliarity score
cosScore_valid, trueScore_valid,res = testVerify(network, verify_loader_valid)
print("Cosscore is:",cosScore_valid.shape,trueScore_valid.shape,len(res))

In [ ]:
# Read in verification pairs for test
verifyData_test = MyDatasetVerify(hyper["verifyTestPairListPath"], hyper["verifyTestImgFolderPath"])
verify_loader_args_test = dict(shuffle=False, batch_size=300, num_workers=2, pin_memory=True)
verify_loader_test = DataLoader(verifyData_test, **verify_loader_args_test)

In [ ]:
cosScore_test, _,result = testVerify(network, verify_loader_test)

In [ ]:
print(len(result))

0


In [ ]:
# Save predictied similarity
import pandas as pd
cosScore_test = np.array(cosScore_test)
np.save("score.npy", cosScore_test)
trial = np.array(verifyData_test.getPairList())
print(trial.shape,cosScore_test.shape)
df = pd.DataFrame({"trial" : trial, "score" : cosScore_test})
df.to_csv("sample.csv", index=False)

(51835,) (51835,)


In [ ]:
51835/173

299.6242774566474